<a href="https://colab.research.google.com/github/m0tchy/camera-geometry-tutorial/blob/main/09_intrinsic_calibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 内部パラメーター（カメラ行列）

https://docs.opencv.org/4.5.0/d9/d0c/group__calib3d.html#details

これまで説明した通り、カメラは姿勢を表す $R$, $\boldsymbol{T}$ によって世界座標の点をカメラ座標に変換します。
$$ 
\boldsymbol{X}^\mathsf{C} = R X^\mathsf{W} + \boldsymbol{T}
$$
同次座標では、
$$
\tilde{\boldsymbol{X}}^\mathsf{C} = \begin{bmatrix} R & \boldsymbol{T} \\ 0^\top & 1 \end{bmatrix} \tilde{\boldsymbol{X}}^\mathsf{W} 
$$
そして、それを透視投影して、投影面 $Z = f$ 上の点に映します。
$$
\tilde{\boldsymbol{X}}'^\mathsf{C}  = \begin{bmatrix} 
    f & 0 & 0 & 0 \\ 
    0 & f & 0 & 0 \\ 
    0 & 0 & f & 0 \\ 
    0 & 0 & 1 & 0 \end{bmatrix} \tilde{\boldsymbol{X}}^\mathsf{C} 
$$
この計算結果の X, Y 座標だけを使って2次元の図形と考えたものがカメラ画像です。
そこで、Z を最初から計算しないようにするとこうなります。
$$
\tilde{\boldsymbol{x}}  = \begin{bmatrix} 
    f & 0 & 0 & 0 \\ 
    0 & f & 0 & 0 \\ 
    0 & 0 & 1 & 0 \end{bmatrix} \tilde{\boldsymbol{X}}^\mathsf{C} 
$$
左辺を $\tilde{\boldsymbol{x}}$ に変えました。これは2次元平面の同次座標です。
ここで、 $f$ はカメラの特性を表すものですが、以下のように、 Z 成分を無視するという計算と分けることができます。

\begin{align}
\tilde{\boldsymbol{x}}  &= 
\begin{bmatrix} 
    f & 0 & 0  \\ 
    0 & f & 0  \\ 
    0 & 0 & 1  \end{bmatrix}
\begin{bmatrix} 
    1 & 0 & 0 & 0 \\ 
    0 & 1 & 0 & 0 \\ 
    0 & 0 & 1 & 0 \end{bmatrix} 
\tilde{\boldsymbol{X}}^\mathsf{C} \\
&=
\begin{bmatrix} 
    f & 0 & 0  \\ 
    0 & f & 0  \\ 
    0 & 0 & 1  \end{bmatrix} 
\begin{bmatrix} I \mid 0 \end{bmatrix} 
\tilde{\boldsymbol{X}}^\mathsf{C} 
\end{align}

カメラの姿勢の計算まで合わせると、
\begin{align}
\tilde{\boldsymbol{x}}  &= 
\begin{bmatrix} 
    f & 0 & 0  \\ 
    0 & f & 0  \\ 
    0 & 0 & 1  \end{bmatrix}
\begin{bmatrix} I \mid 0 \end{bmatrix} 
\begin{bmatrix} R & \boldsymbol{T} \\ 0^\top & 1 \end{bmatrix}\tilde{\boldsymbol{X}}^\mathsf{W} \\
&=
\begin{bmatrix} 
    f & 0 & 0  \\ 
    0 & f & 0  \\ 
    0 & 0 & 1  \end{bmatrix}
\begin{bmatrix} R \mid \boldsymbol{T} \end{bmatrix} 
\tilde{\boldsymbol{X}}^\mathsf{W}
\end{align}
というように、シンプルになります。

さて、3次元の点を計測したとき、最終的には画像上の点に対応しますが、
画像の座標はふつう左上が原点です。また、
カメラの投影中心がちょうど画像の真ん中であるとは限りません。
さらに言えば、カメラの光軸に対して [CCD](https://ja.wikipedia.org/wiki/CCD%E3%82%A4%E3%83%A1%E3%83%BC%E3%82%B8%E3%82%BB%E3%83%B3%E3%82%B5) が正確に垂直に置かれているとも限らず、その傾きによって画像は変形するでしょう。

正規化されたカメラ画像から画素に対する画像座標へ直接変換をするために、線形変換（アフィン変換）で書ける部分と、それ以外の非線形な変換 $g$ に分けることを考えます。

\begin{align}
\tilde{\boldsymbol{x}}  &= g\left( 
\begin{bmatrix} 
    f_x & s & c_x  \\ 
    0 & f_y & c_y  \\ 
    0 & 0 & 1  \end{bmatrix}
\begin{bmatrix} 
    1 & 0 & 0 & 0 \\ 
    0 & 1 & 0 & 0 \\ 
    0 & 0 & 1 & 0 \end{bmatrix} 
\tilde{\boldsymbol{X}}^\mathsf{C} \right) \\
&= g\left(A \left[ R \mid \boldsymbol{T} \right] \tilde{\boldsymbol{X}}^\mathsf{W} \right)
\end{align}
ここで、 $f_x$, $f_y$ は焦点距離が歪みによって x, y 方向に少しずれがある可能性を考慮しています。
また、カメラ座標では光軸が (0, 0) を通りますが、これが画像のどこになるかを表す平行移動成分 $c_x, c_y$ があり、
また $s$ はせん断係数です。
これらを**カメラの内部パラメーター** (intrinsic parameters) といい、
一般にこのような上三角行列を内部行列とかカメラ行列といいます。

そして、このカメラ固有の情報を計測することをカメラの（内部）**キャリブレーション** (calibration; 校正) といいます。

非線形な歪みを表す $g$ は、 radial distortion と tangential distortion によってモデル化することができますが詳細は省略します。
あらかじめ画像の歪みを補正することで、それ以降の計算では気にする必要がなくなります。

# 内部パラメーターのキャリブレーション（内部校正）

[OpenCV のチュートリアル](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_calib3d/py_calibration/py_calibration.html) を参考にしていますが、 Colab でできるように改変しています。



In [ ]:
import cv2

import numpy as np
import matplotlib.pyplot as plt

import os

まず、キャリブレーションに使う画像を撮影しアップロードします。
資料フォルダにある `Calibration Chess Board(A4).pdf` をパソコンなどで表示し、フルスクリーンにします (Acrobat Reader の Windows 版では Ctrl-L)。

その画面を、いろんな距離、いろんな角度で撮影します。
20枚以上はとってください。

それを適当なフォルダにアップロードします。
左の「フォルダ」アイコンをクリックして、ファイルの画面にします。
開いているところを右クリックして「新しいフォルダ」を作り、その中に撮った画像をアップロードしてください。

以下のコードで、そのフォルダを指定すると、ファイルがリストアップされるはずです。
なお、このアップロードしたファイルは Google Colab が切断されると消えてしまうので、やり直すときは再度アップロードする必要があります。

In [ ]:
images_dir = "images"
fn_images = os.listdir(images_dir)
fn_images

うまく取得できたら、画像を読み込んでリストにします。

In [ ]:
images = []

for fn in fn_images:
    images.append(cv2.imread(images_dir + "/" + fn))
    print(fn, images[0].shape)

# 1つのカメラのキャリブレーションなので、
# 画像サイズは全部同じであるはず
# numpy.ndarray では (高さ, 幅, チャンネル) の順なので、
# OpenCV の関数用に入れ替える
image_size = images[0].shape[1::-1]

チェスボードの交点を検出する準備をします。
* 今回は、マス目が 8x6 になっているので、交点は 7x5 です。
* パターンの世界座標を用意します。
* 検出の基準を決めます。

In [ ]:
corner_size = (7, 5)

# チェスボードの交点の世界座標
# チェスボードを Z = 0 平面と考え、
# (0, 0, 0) から (7-1, 5-1, 0)
object_points = np.mgrid[0:corner_size[0], 0:corner_size[1], 0:1].T.reshape(-1, 3).astype(np.float32)
object_points.shape

# 検出のための基準を決める（詳細は最初はわからなくても可）
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


コーナーの検出をします。必ずしも成功するとは限りません。また、それなり時間がかかります。

In [ ]:
%%time

detected = []  # 検出が成功した画像の番号
detected_corners = []  # 検出した結果
object_list = []  # 対応するパターン

for i, img in enumerate(images):

    # 検出のためにグレースケールに変換する
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #plt.imshow(img_gray, cmap="gray")

    ret, corners = cv2.findChessboardCorners(img_gray, corner_size, None)
    #ret, corners.shape

    # 失敗したらスキップ
    if ret != True:
        print(f"failed for the image {i}")
        continue
    
    # 検出の精度を高める
    corners = cv2.cornerSubPix(img_gray, corners, (11,11), (-1,-1), criteria)

    # 結果の保存
    detected.append(i)
    detected_corners.append(corners)
    object_list.append(object_points)

    # 結果の表示
    # plt.figure()
    # plt.imshow(img[:,:,[2,1,0]])
    # plt.scatter(corners[:,0,0], corners[:,0,1], color="RED")


キャリブレーションの関数はいくつかあるが、一番基本的な Zhang の方法を使う場合。

In [ ]:
reproj_error, camera_matrix, dist, rvecs, tvecs = cv2.calibrateCamera(
    objectPoints=object_list,  # 各パターンはすべて同じなので、その回数分リピートする
    imagePoints=detected_corners, 
    imageSize=image_size,
    cameraMatrix=None,  # 内部パラメーターの初期値は無し
    distCoeffs=None  # 歪み係数の初期値は無し
)

print(f"RMS reprojection error: {reproj_error}")
print("intrinsic matrix:")
print(camera_matrix)
print(f"ditortions: {dist}")

座標軸を表示。
チェスボードからの相対的なカメラの姿勢を求めることができ、逆に考えるとカメラから見たチェスボードの座標系を知ったということになるので、
座標軸を計算して表示できます。

本当は歪み補正をしたほうがいいのですが、それは自分でやってみてください。上のURLに説明が書かれています。

In [ ]:
for i in range(len(detected)):
    img_copy = images[detected[i]].copy()
    cv2.drawFrameAxes(img_copy, camera_matrix, dist, rvecs[i], tvecs[i], 10, 30)

    plt.figure()
    plt.imshow(img_copy[:,:,[2,1,0]])


## note

* 求まった $f_x$, $f_y$ の単位は pixel です。もし、物理的な長さにしたい場合は、 CCD の1画素の大きさを掛けることで求まります。例えば、カメラの EXIF には焦点距離が mm で記録されていますが、この値になるはずです。CCD のサイズはカメラの仕様などに載っています。
* `rvec` は回転ベクトルと呼ばれるもので、回転軸の単位方向ベクトル $n$ にその回転角度を掛けたものです。つまり、長さが回転角度になっています。
* 上のコードでは、 `object_points` として、 (0, 1, 0) のような整数座標を与えていました。この場合、世界の物理的な単位は不明です。もし、整数の代わりに画面上のマス目の長さを使えば、 `rvec`, `tvec` は、その長さの単位に関するものに変わります。ただし、内部パラメーターを計算するのが目的の場合は世界の単位は特に関係ありません。



